<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/LearnGrerelAIdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets -q

In [22]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
maxRowtobeselected = 100

# Load the dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

# Filter for the "Retail" domain
retail_dataset = dataset.filter(lambda example: example['domain'] == 'retail')

# Convert to pandas DataFrame for easier manipulation
retail_df = retail_dataset.to_pandas()
print(len(retail_df))


# Assume we want equal distribution based on 'sql_prompt'
# Get unique values in 'sql_prompt' and calculate the number of samples per group
unique_domains = retail_df['sql_prompt'].unique()
print(len(unique_domains))
samples_per_group = maxRowtobeselected // len(unique_domains)

# Stratified sampling using 'sql_prompt' to ensure equal distribution
stratified_df = pd.concat([
    group.sample(n=min(samples_per_group, len(group)), random_state=42)
    for _, group in retail_df.groupby('sql_prompt')
])

# If fewer than maxRowtobeselected rows are collected due to small groups, sample additional rows randomly
if len(stratified_df) < maxRowtobeselected:
    additional_samples = retail_df.drop(stratified_df.index).sample(n=maxRowtobeselected-len(stratified_df), random_state=42)
    stratified_df = pd.concat([stratified_df, additional_samples])

# Convert back to a Hugging Face Dataset
final_dataset = dataset.from_pandas(stratified_df)

# Select specific columns
final_dataset = final_dataset.map(lambda example: {
    'domain': example['domain'],
    'domain_description': example['domain_description'],
    'sql_prompt': example['sql_prompt'],
    'sql': example['sql']
},remove_columns=dataset.column_names)

# Verify the result by printing it
print(final_dataset)
print(final_dataset[0])

979
979


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['domain', 'domain_description', 'sql_prompt', 'sql', '__index_level_0__'],
    num_rows: 100
})
{'domain': 'retail', 'domain_description': 'Retail data on circular supply chains, ethical labor practices, product transparency, and consumer education.', 'sql_prompt': 'Determine the number of products manufactured using sustainable materials', 'sql': 'SELECT COUNT(*) as num_products_with_sustainable_materials FROM product_materials WHERE is_sustainable = TRUE;', '__index_level_0__': 199}


In [3]:
from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split = "train")
# Filter for "Retail" domain, select first 50 rows, and choose specific columns
retail_dataset = dataset.filter(lambda example: example['domain'] == 'retail') \
                        .select(range(50)) \
                        .map(lambda example: {'domain': example['domain'],
                                              'domain_description': example['domain_description'],
                                              'sql_prompt': example['sql_prompt'],
                                              'sql': example['sql']})#,
                             #remove_columns=dataset.column_names)  # Remove all original columns
print(retail_dataset)
print(retail_dataset[0])

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 50
})
{'id': 5145, 'domain': 'retail', 'domain_description': 'Retail data on circular supply chains, ethical labor practices, product transparency, and consumer education.', 'sql_complexity': 'subqueries', 'sql_complexity_description': 'subqueries, including correlated and nested subqueries', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': 'What is the percentage of products that are made from recycled materials?', 'sql_context': 'CREATE TABLE products(product_id INT, is_recycled BOOLEAN);', 'sql': 'SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products)) as percentage FROM products WHERE is_recycled = TRUE;', 'sql_explanation': 'This query calculates the percenta

In [ ]:
# Iterate over the dataset and print each row in a readable format
for i, row in enumerate(retail_dataset):
    print(f"Row {i+1}:")
    for key, value in row.items():
        print(f"  {key}: {value}")
    print("-" * 20)

Row 1:
  id: 5145
  domain: retail
  domain_description: Retail data on circular supply chains, ethical labor practices, product transparency, and consumer education.
  sql_complexity: subqueries
  sql_complexity_description: subqueries, including correlated and nested subqueries
  sql_task_type: analytics and reporting
  sql_task_type_description: generating reports, dashboards, and analytical insights
  sql_prompt: What is the percentage of products that are made from recycled materials?
  sql_context: CREATE TABLE products(product_id INT, is_recycled BOOLEAN);
  sql: SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM products)) as percentage FROM products WHERE is_recycled = TRUE;
  sql_explanation: This query calculates the percentage of products that are made from recycled materials by dividing the count of recycled products by the total number of products.
--------------------
Row 2:
  id: 5352
  domain: retail
  domain_description: Retail data on circular supply chains, ethical la